## 💡 1. Introduction 

[TODO]

## ⚙️ 2. Settings

In [1]:
GAME_DETAILS_PATH = '../data/game_details.json'
GAME_REVIEWS_PATH = '../data/game_reviews.json'

GAME_DETAILS_SELECTION = ['title', 'platform', 'publisher', 'genres']
GAME_REVIEWS_SELECTION = ['title', 'platform', 'meta_score', 'review_critic']

## 📚 3. Preperations

### 3.1 Importing libraries

In [2]:
import os
import io
import math
import base64

import numpy as np
import pandas as pd
import seaborn as sns

from PIL import Image
import matplotlib.pyplot as plt

### 3.2 Loading the data

In [3]:
games = pd.read_json(GAME_DETAILS_PATH)
games = games[GAME_DETAILS_SELECTION]
games.sample(n=5, random_state=42)

,title,platform,publisher,genres
3571,Red Orchestra: Ostfront 41-45,PC,Bold Games,"[Action, Shooter, Shooter, First-Person, Histo..."
16592,Dreamcast Collection,Xbox 360,Sega,"[Miscellaneous, Compilation, Compilation]"
16533,Planar Conquest,PC,Wastelands Interactive,"[Strategy, Turn-Based, 4X]"
5245,Conker: Live & Reloaded,Xbox,Microsoft Game Studios,"[Action, Shooter, Third-Person, Fantasy]"
16339,Port Royale 3: Pirates and Merchants,PC,Kalypso,"[Strategy, Real-Time, General, General]"


In [4]:
ratings = pd.read_json(GAME_REVIEWS_PATH)
ratings = ratings[GAME_REVIEWS_SELECTION]
ratings.sample(n=5, random_state=42)

,title,platform,meta_score,review_critic
88541,Terraria,PC,85.0,GameTrailers
108679,Virtual Pool 3,PC,85.0,GamerWeb PC
393766,Tecmo Classic Arcade,Xbox,59.0,Xbox Evolved
173811,Far Cry 5,PC,80.0,Game World Navigator Magazine
372616,Legends of Wrestling II,PlayStation 2,50.0,Gaming Age


### 3.3 Loading the images

## 🧹 4. Cleaning

### 4.1 Solving null values

In [5]:
games.isnull().sum()

title         0
platform      0
publisher    12
genres        0
dtype: int64

In [6]:
ratings = ratings[ratings['meta_score'].notna()]
ratings.isnull().sum()

title            0
platform         0
meta_score       0
review_critic    1
dtype: int64

### 4.2 Solving duplicate values

In [7]:
games['genres'] = games['genres'].apply(lambda x: list(set(x)))
games = games.drop_duplicates(subset=['title', 'platform'])

In [8]:
ratings = ratings.drop_duplicates()
ratings

,title,platform,meta_score,review_critic
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,100.0,Hot Games
1,The Legend of Zelda: Ocarina of Time,Nintendo 64,100.0,GameShark
2,The Legend of Zelda: Ocarina of Time,Nintendo 64,100.0,Cincinnati Enquirer
3,The Legend of Zelda: Ocarina of Time,Nintendo 64,100.0,Nintendophiles
4,The Legend of Zelda: Ocarina of Time,Nintendo 64,100.0,Electronic Gaming Monthly
...,...,...,...,...
426390,Family Party: 30 Great Games Obstacle Arcade,Wii U,10.0,Nintendo Life
426391,Family Party: 30 Great Games Obstacle Arcade,Wii U,10.0,NintendoWorldReport
426392,Family Party: 30 Great Games Obstacle Arcade,Wii U,10.0,Gamereactor Denmark
426393,Family Party: 30 Great Games Obstacle Arcade,Wii U,10.0,Eurogamer Sweden


## ♻️ 5. Preprocessing

### 5.1 Rating distribution

## 💾 6. Storage

In [9]:
%store games
%store ratings

Stored 'games' (DataFrame)
Stored 'ratings' (DataFrame)
